In [ ]:
import os
import arcpy
from arcpy.sa import Con, Raster

in_dir = r"Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\NBR_old"
out_dir = os.path.join(in_dir, "shp")
os.makedirs(out_dir, exist_ok=True)

arcpy.env.workspace = in_dir
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

arcpy.CheckOutExtension("Spatial")

# Use scratch geodatabase for temp rasters (avoids in_memory locks)
scratch_gdb = arcpy.env.scratchGDB  # e.g., ...\AppData\Local\Temp\ArcGISProTemp...\scratch.gdb

try:
    tifs = arcpy.ListRasters("NBR_*_*.tif", "TIF")
    if not tifs:
        raise RuntimeError(f"No rasters found in {in_dir} matching NBR_*_*.tif")

    for tif_name in tifs:
        in_ras = os.path.join(in_dir, tif_name)
        base = os.path.splitext(tif_name)[0]
        out_shp = os.path.join(out_dir, base + ".shp")

        # temp raster name in scratchGDB (must be a valid dataset name)
        tmp_name = f"cls_{base}"
        # filegdb name length limit is 160; keep it short
        if len(tmp_name) > 150:
            tmp_name = "cls_" + base[-140:]
        tmp_ras = os.path.join(scratch_gdb, tmp_name)

        try:
            # Build classified raster:
            # <= 0.4 -> 1
            # <= 0.5 -> 2
            # else   -> 3
            r = Raster(in_ras)
            cls = Con(r <= 0.4, 1,
                      Con(r <= 0.5, 2, 3))

            # Save to scratch GDB
            cls.save(tmp_ras)

            # Important: drop references to release locks
            del cls
            del r

            # Polygonize (NO simplification)
            arcpy.conversion.RasterToPolygon(
                in_raster=tmp_ras,
                out_polygon_features=out_shp,
                simplify="NO_SIMPLIFY",
                raster_field="Value"
            )

            # Add a nicer "class" field copied from GRIDCODE
            fieldnames = {f.name.lower() for f in arcpy.ListFields(out_shp)}
            if "class" not in fieldnames:
                arcpy.management.AddField(out_shp, "class", "SHORT")
                arcpy.management.CalculateField(out_shp, "class", "!gridcode!", "PYTHON3")

            print(f"OK: {tif_name} -> {out_shp}")

        finally:
            # Best-effort cleanup of temp raster
            try:
                if arcpy.Exists(tmp_ras):
                    arcpy.management.Delete(tmp_ras)
            except arcpy.ExecuteError:
                # If it still locks occasionally, don't crash the whole batch
                arcpy.AddWarning(f"Could not delete temp raster: {tmp_ras} (locked)")

finally:
    arcpy.CheckInExtension("Spatial")

print("DONE.")


In [ ]:
# ArcGIS Pro (arcpy) - Clip all shapefiles by a mask polygon layer
# Input SHP folder:  Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\NBR_to_map\shp
# Mask SHP:          Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\grass_mask.shp
# Output folder:     Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\NBR_to_map\shp_masked
# Output names:      same as inputs

import os
import arcpy

in_dir = r"Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\NBR_old\shp"
mask_shp = r"Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\grass_mask.shp"
out_dir = r"Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\NBR_old\shp_masked"

os.makedirs(out_dir, exist_ok=True)

arcpy.env.workspace = in_dir
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

if not arcpy.Exists(mask_shp):
    raise FileNotFoundError(f"Mask not found: {mask_shp}")

# List all shapefiles in the input folder
shps = arcpy.ListFeatureClasses(feature_type="Polygon")  # your outputs from RasterToPolygon are polygons
if not shps:
    # fallback: list everything in case feature_type filter doesn't match
    shps = arcpy.ListFeatureClasses()

if not shps:
    raise RuntimeError(f"No shapefiles found in: {in_dir}")

for shp in shps:
    in_fc = os.path.join(in_dir, shp)
    out_fc = os.path.join(out_dir, shp)  # keep same name

    # Skip non-shapefile feature classes if any appear
    if not shp.lower().endswith(".shp"):
        continue

    # Clip keeps only parts inside the mask polygons
    arcpy.analysis.Clip(
        in_features=in_fc,
        clip_features=mask_shp,
        out_feature_class=out_fc
    )

    print(f"OK: {shp} -> {out_fc}")

print("DONE.")


In [ ]:
# ArcGIS Pro (arcpy) - Remove polygons where class = 3 from all shapefiles
# Input folder:   Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\NBR_to_map\shp_masked
# Output folder:  Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\NBR_to_map\shp_only_mapping
# Output names:   same as inputs

import os
import arcpy

in_dir = r"Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\NBR_old\shp_masked"
out_dir = r"Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\NBR_old\shp_only_mapping"
os.makedirs(out_dir, exist_ok=True)

arcpy.env.workspace = in_dir
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

shps = arcpy.ListFeatureClasses()
if not shps:
    raise RuntimeError(f"No shapefiles found in: {in_dir}")

for shp in shps:
    if not shp.lower().endswith(".shp"):
        continue

    in_fc = os.path.join(in_dir, shp)
    out_fc = os.path.join(out_dir, shp)

    # Prefer field "class", but fallback to "gridcode" if needed
    fields = {f.name.lower() for f in arcpy.ListFields(in_fc)}
    if "class" in fields:
        class_field = "class"
    elif "gridcode" in fields:
        class_field = "gridcode"
    else:
        raise RuntimeError(f"{shp}: no 'class' or 'gridcode' field found.")

    # Keep only polygons where class != 3
    where = f"{arcpy.AddFieldDelimiters(in_fc, class_field)} <> 3"

    arcpy.analysis.Select(
        in_features=in_fc,
        out_feature_class=out_fc,
        where_clause=where
    )

    print(f"OK: {shp} -> {out_fc}")

print("DONE.")


In [ ]:
import arcpy
import os

# ----------------------------
# USER INPUTS
# ----------------------------
in_fc = r"Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\Full_diebacks.shp"   # or shapefile path
out_fc = r"Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\Full_diebacks_union.shp"
count_field = "OVLP_CNT"

# Optional: where to create temp layers
scratch_gdb = arcpy.env.scratchGDB

# ----------------------------
# ENV
# ----------------------------
arcpy.env.overwriteOutput = True

# ----------------------------
# 1) UNION -> create non-overlapping pieces
# ----------------------------
# Union of a single feature class self-unions and splits overlaps
arcpy.analysis.Union([in_fc], out_fc, "ALL", gaps="GAPS")

# ----------------------------
# 2) Add overlap count field
# ----------------------------
existing = {f.name.upper() for f in arcpy.ListFields(out_fc)}
if count_field.upper() not in existing:
    arcpy.management.AddField(out_fc, count_field, "LONG")

# ----------------------------
# 3) Count overlaps using Spatial Join
#    Target = union pieces
#    Join = original polygons
#    JOIN_ONE_TO_ONE with KEEP_ALL produces Join_Count per piece
# ----------------------------
sj_fc = os.path.join(scratch_gdb, "tmp_sj_overlap_count")

# If it already exists in scratch from a previous run
if arcpy.Exists(sj_fc):
    arcpy.management.Delete(sj_fc)

arcpy.analysis.SpatialJoin(
    target_features=out_fc,
    join_features=in_fc,
    out_feature_class=sj_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    match_option="INTERSECT"
)

# ----------------------------
# 4) Transfer Join_Count -> OVLP_CNT back to union output
#    We'll join by OID to avoid geometry matching issues.
# ----------------------------
oid_union = arcpy.Describe(out_fc).OIDFieldName
oid_sj = arcpy.Describe(sj_fc).OIDFieldName  # OID of sj_fc itself
# SpatialJoin output preserves target OID in a field named like TARGET_FID.
# In Pro, it's typically "TARGET_FID" for shapefile targets,
# or "TARGET_FID" / "ORIG_FID" depending on tools/version.
# We'll detect it safely.

sj_fields = [f.name for f in arcpy.ListFields(sj_fc)]
target_link_field = None
for cand in ("TARGET_FID", "TARGET_FID_1", "ORIG_FID", "FID", "FID_" + os.path.basename(out_fc)):
    if cand in sj_fields:
        target_link_field = cand
        break

if target_link_field is None:
    # last resort: find any field containing "TARGET" and "FID"
    for f in sj_fields:
        if "TARGET" in f.upper() and "FID" in f.upper():
            target_link_field = f
            break

if target_link_field is None:
    raise RuntimeError(
        "Could not find the target-link field (e.g., TARGET_FID) in SpatialJoin output. "
        "Open sj_fc in Fields view and check the field that references the union target OID."
    )

# Build a dictionary: union_oid -> join_count
count_dict = {}
with arcpy.da.SearchCursor(sj_fc, [target_link_field, "Join_Count"]) as sc:
    for u_oid, jcnt in sc:
        # u_oid can be -1 for gaps; keep them as 0 overlaps if desired
        if u_oid is None or u_oid == -1:
            continue
        count_dict[int(u_oid)] = int(jcnt)

# Update out_fc overlap field
with arcpy.da.UpdateCursor(out_fc, [oid_union, count_field]) as uc:
    for u_oid, _ in uc:
        uc.updateRow([u_oid, count_dict.get(int(u_oid), 0)])

# Clean up temp
arcpy.management.Delete(sj_fc)

print("Done.")
print(f"Output: {out_fc}")
print(f"Overlap count field: {count_field}")
